In [50]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import (
    Input,
    Activation,
    Add,
    Dense,
    Conv2D,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from keras.layers import BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Model
from keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix

In [51]:
test_dir = "dataset/test"
train_dir = "dataset/train"

In [52]:
print(os.listdir(test_dir))
print(os.listdir(train_dir))

['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']


In [53]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.25,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode="nearest",
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [54]:
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(84, 84), batch_size=16, class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(84, 84), batch_size=16, class_mode="categorical"
)

Found 11643 images belonging to 10 classes.
Found 4997 images belonging to 10 classes.


In [55]:
def residual_block(X, kernel_size, filters, reduce=False, stride=2):
    """
    Implement a residual block for ResNet architectures.

    Arguments:
    X           -- input tensor of shape (m, height, width, channels)
    kernel_size -- integer, kernel size of the middle convolutional layer in the main path
    filters     -- python list of integers, defining the number of filters in the CONV layers of the main path
    reduce      -- boolean, whether to reduce the spatial dimensions and increase depth;
                    if True, applies 1x1 CONV layer to the shortcut path.
    stride      -- integer, strides for the convolutional layer

    Returns:
    X           -- output of the identity block, tensor of shape (height, width, channels)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We will need this later to add back to the main path.
    X_shortcut = X

    if reduce:
        # if we are to reduce the spatial size, apply a 1x1 CONV layer to the shortcut path
        X = Conv2D(
            filters=F1,
            kernel_size=(1, 1),
            strides=(stride, stride),
            padding="valid",
            kernel_initializer="he_normal",
        )(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation("relu")(X)

        X_shortcut = Conv2D(
            filters=F3,
            kernel_size=(1, 1),
            strides=(stride, stride),
            padding="valid",
            kernel_initializer="he_normal",
        )(X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)
    else:
        # First component of main path
        X = Conv2D(
            filters=F1,
            kernel_size=(1, 1),
            strides=(1, 1),
            padding="valid",
            kernel_initializer="he_normal",
        )(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation("relu")(X)

    # Second component of main path
    X = Conv2D(
        filters=F2,
        kernel_size=(kernel_size, kernel_size),
        strides=(1, 1),
        padding="same",
        kernel_initializer="he_normal",
    )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)

    # Third component of main path
    X = Conv2D(
        filters=F3,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding="valid",
        kernel_initializer="he_normal",
    )(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X

In [56]:
def ResNet_50(input_shape, classes):
    """
    Arguments:
    input_shape -- tuple shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Block 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer="he_normal")(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Block 2
    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    # Block 3
    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])

    # Block 4
    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])

    # Block 5
    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])

    # Global Average Pooling to reduce spatial dimensions
    X = GlobalAveragePooling2D()(X)

    # Fully Connected Layer for classification
    X = Dense(classes, activation="softmax")(X)

    # Create the model
    model = Model(inputs=X_input, outputs=X, name="ResNet50")

    return model

In [57]:
def Modified_ResNet50(input_shape, classes):
    """
    Arguments:
    input_shape -- tuple shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer="he_normal")(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = residual_block(X, 3, [64, 64, 256], reduce=True, stride=1)
    X = residual_block(X, 3, [64, 64, 256])
    X = residual_block(X, 3, [64, 64, 256])

    # Stage 3
    X = residual_block(X, 3, [128, 128, 512], reduce=True, stride=2)
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])
    X = residual_block(X, 3, [128, 128, 512])

    # Stage 4
    X = residual_block(X, 3, [256, 256, 1024], reduce=True, stride=2)
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])
    X = residual_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = residual_block(X, 3, [512, 512, 2048], reduce=True, stride=2)
    X = residual_block(X, 3, [512, 512, 2048])
    X = residual_block(X, 3, [512, 512, 2048])

    # Global Average Pooling to reduce spatial dimensions
    X = GlobalAveragePooling2D()(X)

    # Add Dropout to prevent overfitting
    X = Dropout(0.5)(X)

    # Fully Connected Layer for classification
    X = Dense(classes, activation="softmax")(X)

    # Create the model
    model = Model(inputs=X_input, outputs=X, name="Modified_ResNet50")

    return model

In [58]:
# Define the shape of the input images and number of classes
input_shape = (84, 84, 3)
num_classes = 10

# Initialize the modified ResNet50 model with the specified parameters
modified_resnet50_model = Modified_ResNet50(
    input_shape=input_shape, classes=num_classes
)

In [59]:
plot_model(modified_resnet50_model, show_shapes=True, show_layer_names=False, dpi=120)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [60]:
modified_resnet50_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [61]:
# Add ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=15, min_lr=0.00001
)

# Add EarlyStopping callback
early_stopping = EarlyStopping(
    monitor="val_loss", mode="min", patience=50, restore_best_weights=True, verbose=1
)

In [62]:
num_classes = len(train_generator.class_indices)  # Auto-detect class count
print(f"Detected {num_classes} classes.")

Detected 10 classes.


In [ ]:
# Total number of epochs
num_epochs = 200

# Train the model
history = modified_resnet50_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[reduce_lr, early_stopping],
)

Epoch 1/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 435s 555ms/step - accuracy: 0.3323 - loss: 2.7429 - val_accuracy: 0.2598 - val_loss: 2.8554 - learning_rate: 0.0010
Epoch 2/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 393s 540ms/step - accuracy: 0.4808 - loss: 1.6393 - val_accuracy: 0.3158 - val_loss: 5.5600 - learning_rate: 0.0010
Epoch 3/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 397s 546ms/step - accuracy: 0.4500 - loss: 1.8064 - val_accuracy: 0.4319 - val_loss: 1.7530 - learning_rate: 0.0010
Epoch 4/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 395s 543ms/step - accuracy: 0.4695 - loss: 1.7066 - val_accuracy: 0.5125 - val_loss: 1.9334 - learning_rate: 0.0010
Epoch 5/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 391s 537ms/step - accuracy: 0.4618 - loss: 1.7390 - val_accuracy: 0.3386 - val_loss: 12.2969 - learning_rate: 0.0010
Epoch 6/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 398s 547ms/step - accuracy: 0.5153 - loss: 1.4960 - val_accuracy: 0.4437 - val_loss: 1.8541 - learning_rate: 0.0010
Epoch 7/200
728/728 ━━━━━━━━━━━━━━━━━━━━ 395s 543ms/step 

In [ ]:
test_loss, test_accuracy = modified_resnet50_model.evaluate(test_generator)
print(f"Test accuracy : {test_accuracy:.2f}")